<a href="https://colab.research.google.com/github/PhD-Gabriel-Caballero/ESA-summer-school-2023-Croatia/blob/main/Multi_crop_land_cover_classification_based_on_S1_time_series_using_machine_learning_algorithms_in_GEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Multi-crop land cover classification based on S1 time series using machine learning algorithms in GEE

In [ ]:
!pip install geemap

**Import libraries**

In [2]:
import ee
import geemap

**Mount Google Drive**

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Create an interactive map

In [3]:
Map = geemap.Map()
Map

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=bEl4cy68ZFMqpEdnG3DEtiQ2x0Ck80w-9NylZwXDKVU&tc=_5PxGiPmWLSMtXSVbkIw9Yha0Yh4BPrGUvvDrGBFAu8&cc=g4RNNV5Nr7c8NU3D6tB0L4VI0hHWivKDqVClw3R5bjg

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BVhtaSoDFIhZha3suvibayNaRlC1CswPGkgpPFZLFbHS2boQ3QR6UA

Successfully saved authorization token.


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

**Add data to the map**

In [4]:
# point = ee.Geometry.Point([-122.4439, 37.7538])
point = ee.Geometry.Point([-87.7719, 41.8799])

image = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
    .filterBounds(point)
    .filterDate('2019-01-01', '2019-12-31')
    .sort('CLOUD_COVER')
    .first()
    .select('B[1-7]')
)

vis_params = {'min': 0, 'max': 3000, 'bands': ['B5', 'B4', 'B3']}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")

**Check image properties**

In [5]:
props = geemap.image_props(image)
props.getInfo()
props.get('IMAGE_DATE').getInfo()
props.get('CLOUD_COVER').getInfo()

0.03

In [6]:
# Make the training dataset.
training = image.sample(
    **{
        #     'region': region,
        'scale': 30,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(training, {}, 'training', False)
Map

Map(bottom=24657.0, center=[41.8799, -87.7719], controls=(WidgetControl(options=['position', 'transparent_bg']…

**Train the clusterer**

In [7]:
# Instantiate the clusterer and train it.
n_clusters = 5
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

**Classify the image**

In [8]:
# Cluster the input using the trained clusterer.
result = image.cluster(clusterer)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'clusters')
Map

Map(bottom=24692.0, center=[41.73893835657947, -87.81341558878911], controls=(WidgetControl(options=['position…

**Label the clusters**

In [9]:
legend_keys = ['One', 'Two', 'Three', 'Four', 'ect']
legend_colors = ['#8DD3C7', '#FFFFB3', '#BEBADA', '#FB8072', '#80B1D3']

# Reclassify the map
result = result.remap([0, 1, 2, 3, 4], [1, 2, 3, 4, 5])

Map.addLayer(
    result, {'min': 1, 'max': 5, 'palette': legend_colors}, 'Labelled clusters'
)
Map.add_legend(
    legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright'
)
Map

Map(bottom=24635.0, center=[41.97174336327968, -86.0446904089521], controls=(WidgetControl(options=['position'…

**Visualize the result**

In [10]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

**Export the result to Google Drive**

[S1 time series crop classification](https://mortcanty.github.io/src/s1class.html)

In [12]:
%matplotlib inline
#import ee
#from auxil.eeWishart import omnibus
ee.Initialize()
poly = ee.Geometry({'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-105.10328600000001, 50.24327999999998],
                                                                           [-104.66649400000001, 50.24327999999998],
                                                                           [-104.66649400000001, 50.46604134048255],
                                                                           [-105.10328600000001, 50.46604134048255],
                                                                           [-105.10328600000001, 50.24327999999998]]]})
poly = ee.Geometry(poly)
collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
                      .filterBounds(poly) \
                      .filterDate(ee.Date('2017-03-01'), ee.Date('2017-11-01')) \
                      .filter(ee.Filter.eq('transmitterReceiverPolarisation', ['VV','VH'])) \
                      .filter(ee.Filter.eq('resolution_meters', 10)) \
                      .filter(ee.Filter.eq('instrumentMode', 'IW')) \
                      .filter(ee.Filter.eq('relativeOrbitNumber_start',5)) \
                      .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
collection = collection.sort('system:time_start')

collection.size().getInfo()

19

In [22]:
import math

def get_vvvh(image):
    ''' get 'VV' and 'VH' bands from sentinel-1 imageCollection and restore linear signal from db-values '''
    return image.select('VV','VH').multiply(ee.Image.constant(math.log(10.0)/10.0)).exp()

timeseries_raw = collection \
            .map(get_vvvh) \
            .toBands() \
            .clip(poly) \
            .float()

timeseries_raw.bandNames().length().getInfo()

38

In [23]:
crop2017 = ee.ImageCollection('AAFC/ACI') \
    .filter(ee.Filter.date('2017-01-01', '2017-12-01')) \
    .first() \
    .clip(timeseries_raw.geometry())\
    .float()
labeled_timeseries_raw = ee.Image.cat(timeseries_raw,crop2017)

labeled_timeseries_raw.bandNames().length().getInfo()

39

In [16]:
drexport1 = ee.batch.Export.image.toDrive(labeled_timeseries_rl,
                  description='driveExportTask',
                  folder = 'EarthEngineImages',
                  fileNamePrefix='labeled_timeseries_rl',scale=30,maxPixels=1e10)
drexport1.start()
drexport2 = ee.batch.Export.image.toDrive(labeled_timeseries_raw,
                  description='driveExportTask',
                  folder = 'EarthEngineImages',
                  fileNamePrefix='labeled_timeseries_raw',scale=30,maxPixels=1e10)
drexport2.start()

In [19]:
!ls -l imagery/regina

ls: cannot access 'imagery/regina': No such file or directory


In [18]:
%run scripts/dispms -f imagery/regina/labeled_timeseries_rl.tif  -p [21,23,25] \
-F imagery/regina/labeled_timeseries_rl.tif -E 2 -P [39,39,39]

Exception: ignored